# STE Binary Network

In [5]:
# import torch base code
import sys
from os import path
sys.path.append(path.join("..", "src"))
import base_torch

## Data

In [ ]:
# imports
from base_torch.process_data import MfccDataset

# Dataset
train_dataset = MfccDataset(
    mfcc_dir=''
)